# `tensorflow.keras` convnet from scratch

This notebook demonstrates the use of a simple convolutional network. On a 2-class classification task--that is, an ouput layer with two nodes activated via `softmax`. Accordingly, loss is calculated using the `categorical cross-entropy`.

References:

* The github repository from the BAGLS team contained in this [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [1]:
# dev convenience
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" # use only this GPU device
os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "03-simple-convnet.ipynb"

PROJECT_NAME = 'bagls-sh-project'
RUN_NAME = 'convnet_from_scratch'
METRICS_TABLE_NAME = 'metrics_table'
GRADCAM_LAYER_NAME = "conv2d_3"

In [3]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# # manage logs
import logging

logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)

W&B:  0.13.5


In [4]:
# tf loader
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7016468560932725208
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 5369884234142610166
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 13448997929073525861
physical_device_desc: "device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1"
]


2022-12-03 20:18:51.348541: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 20:18:53.109537: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-12-03 20:18:53.111419: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1


In [5]:
import config
configs = config.nb_configs

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
    preprocessing_function=None,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

test_datagen = ImageDataGenerator(
    preprocessing_function=None, 
    rescale=configs["rescale"],
)

In [7]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 50018 images belonging to 2 classes.
Found 2632 images belonging to 2 classes.
Found 2300 images belonging to 2 classes.


In [8]:
from tensorflow import keras
from tensorflow.keras import layers

def define_model():
    inputs = keras.Input(shape=(*image_size, 3))
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs['dropout_rate'])(x)
    outputs = layers.Dense(units=2, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
tf.keras.backend.clear_session()
model = define_model()

2022-12-03 20:18:57.309516: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-12-03 20:18:57.309948: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1


In [10]:
from tensorflow.keras import metrics
import custom_metrics
thresh = configs["thresh"]
metrics_dict = {
    "ACC":  metrics.CategoricalAccuracy(name="ACC"),
    "AUC-ROC": custom_metrics.MulticlassAUC(name='ROC', curve='ROC', pos_label=1),
    "AUC-PR": custom_metrics.MulticlassAUC(name='PR', curve='PR', pos_label=1),
    "TP": custom_metrics.MulticlassTruePositives(name="TP", pos_label=1, thresholds=thresh),
    "TN": custom_metrics.MulticlassTrueNegatives(name="TN", pos_label=1, thresholds=thresh),
    "FP": custom_metrics.MulticlassFalsePositives(name="FP", pos_label=1, thresholds=thresh),
    "FN": custom_metrics.MulticlassFalseNegatives(name="FN", pos_label=1, thresholds=thresh),
}

In [11]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses

# opt = optimizers.Adam(learning_rate=1e-06)
opt = optimizers.Adam()
met = list(metrics_dict.values())

model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer=opt,
    metrics=met,
)

In [12]:
# verify arch
# base predictions with untrained classif head
base_preds = model.predict(test_dataset)
base_preds

2022-12-03 20:18:59.073816: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-03 20:19:00.990132: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


array([[0.49660176, 0.50339824],
       [0.4896203 , 0.51037973],
       [0.49598914, 0.5040109 ],
       ...,
       [0.49343473, 0.50656533],
       [0.48994914, 0.51005083],
       [0.49814507, 0.5018549 ]], dtype=float32)

In [13]:
from interpretation import ValLog, GRADCamLogger
# initialize run
run = wandb.init(
    project=PROJECT_NAME, 
    name=RUN_NAME,
    config=configs, 
    job_type='train',
)

wandb_callback = wandb.keras.WandbCallback(
    monitor="val_ROC",
    mode="max",
    save_model=True,
    save_graph=True,
    compute_flops=True,
)

callbacks = [
    wandb_callback,
    ValLog(generator=val_dataset, num_log_batches=1),
#     GRADCamLogger(generator=test_dataset, layer_name=GRADCAM_LAYER_NAME, 
#                   num_log_batches=3)
    
]

In [14]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=configs["epochs"], 
    shuffle=True,
    callbacks=callbacks,
)
run.finish()

2022-12-03 20:19:31.116172: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2022-12-03 20:19:31.116377: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-12-03 20:19:31.121748: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-12-03 20:19:31.122291: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10419 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1
2022-12-03 20:19:31.126512: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/10
391/391 [==============================] - 902s 2s/step - loss: 0.6111 - ACC: 0.6874 - ROC: 0.6534 - PR: 0.5416 - TP: 4762.0000 - TN: 29620.0000 - FP: 2633.0000 - FN: 13003.0000 - val_loss: 0.5676 - val_ACC: 0.7067 - val_ROC: 0.7272 - val_PR: 0.6211 - val_TP: 313.0000 - val_TN: 1547.0000 - val_FP: 150.0000 - val_FN: 622.0000


2022-12-03 20:34:35.625613: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221203_201914-2r03a2cj/files/model-best/assets
Epoch 2/10
391/391 [==============================] - 822s 2s/step - loss: 0.5345 - ACC: 0.7269 - ROC: 0.7695 - PR: 0.6624 - TP: 7932.0000 - TN: 28425.0000 - FP: 3828.0000 - FN: 9833.0000 - val_loss: 0.4730 - val_ACC: 0.7610 - val_ROC: 0.8369 - val_PR: 0.7555 - val_TP: 432.0000 - val_TN: 1571.0000 - val_FP: 126.0000 - val_FN: 503.0000
INFO:tensorflow:Assets written to: /home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221203_201914-2r03a2cj/files/model-best/assets
Epoch 3/10
391/391 [==============================] - 814s 2s/step - loss: 0.4495 - ACC: 0.7767 - ROC: 0.8508 - PR: 0.7655 - TP: 11003.0000 - TN: 27845.0000 - FP: 4408.0000 - FN: 6762.0000 - val_loss: 0.3733 - val_ACC: 0.8222 - val_ROC: 0.9038 - val_PR: 0.8483 - val_TP: 630.0000 - val_TN: 1534.0000 - val_FP: 163.0000 - val_FN: 305.0000
INFO:tensorflow:Assets written to: /home/md

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## End